# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## Setup

First, let's install the necessary packages and connect to the database.

In [1]:
# Install required packages
!pip install ipython-sql==0.4.1 sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db

/home/christopher/code/datasci223_assignment3/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


Connecting to 'sqlite:///sql-murder-mystery.db'

## The Crime

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. Start by retrieving the corresponding crime scene report from the police department’s database.

In [2]:
%%sql

SELECT * 
FROM crime_scene_report
WHERE date = 20180115 
  AND type = 'murder'
  AND city = 'SQL City';

Running query in 'sqlite:///sql-murder-mystery.db'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


**Crime Scene Report Output:**
```
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".
```

Now, let's find the witnesses based on the report.

### Find the Witnesses

In [3]:
%%sql

SELECT * 
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [4]:
%%sql

SELECT * 
FROM person
WHERE name LIKE 'Annabel%'
  AND address_street_name = 'Franklin Ave';

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


**Witness Information:**
*   Witness 1: Morty Schapiro (ID: 14887), lives at 4919 Northwestern Dr.
*   Witness 2: Annabel Miller (ID: 16371), lives at 103 Franklin Ave.

Let's see what they told the police in their interviews.

### Read the Interviews

In [5]:
%%sql

SELECT * 
FROM interview
WHERE person_id IN (14887, 16371);

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


**Interview Transcripts:**
*   **Morty Schapiro (ID 14887):** "I heard a gunshot and then saw a man run out. He had a 'Get Fit Now' bag. The membership number on the bag started with '48Z'. Only gold members have those bags. The man got into a car with a plate that included 'H42W'."
*   **Annabel Miller (ID 16371):** "I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."

Let's combine these clues to find the suspect.

### Follow the Clues

In [6]:
%%sql

SELECT p.id, p.name, p.license_id
FROM person p
JOIN drivers_license dl ON p.license_id = dl.id
JOIN get_fit_now_member gfnm ON p.id = gfnm.person_id
JOIN get_fit_now_check_in gfnc ON gfnm.id = gfnc.membership_id
WHERE gfnm.membership_status = 'gold'
  AND gfnm.id LIKE '48Z%'
  AND dl.plate_number LIKE '%H42W%'
  AND gfnc.check_in_date = 20180109;

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id
67318,Jeremy Bowers,423327


**Suspect Identified:**
The query returns one person:
*   ID: 67318
*   Name: Jeremy Bowers
*   License ID: 423327

Jeremy Bowers matches all the clues from both witnesses.

## Check the Solution

Now we insert the name into the `solution` table to verify.

In [7]:
%%sql

INSERT INTO solution VALUES (1, 'Jeremy Bowers');

SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


**Result:**
```
Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.
```